In [45]:
import requests
import random
import json

url     = 'http://localhost:5005/webhooks/rest/webhook'
DATA    = {
    "message"   : "How do you treat peach leaf curl?", 
    "sender"    : str(random.randint(0, 9))
}

r = requests.post(url, json = DATA)

print(json.loads(r.text))

[{'recipient_id': '0', 'text': 'Extracted slots</br>[Format: (<i>relation</i>) <strong>entity</strong> - <strong>value</strong>]:</br>Group 1:</br>(<i>pest</i>) <strong>descr</strong> - <strong>peach</strong></br>(<i>pest</i>) <strong>name</strong> - <strong>leaf curl</strong></br>(<i>pest</i>) <strong>action</strong> - <strong>treat</strong></br></br>Composed 1 additional queries:</br>1) <i>peach leaf curl treat</i></br>'}, {'recipient_id': '0', 'text': 'Results with slots improvement... Top 5 results.'}, {'recipient_id': '0', 'custom': {'text': 'Here are my top results:', 'payload': 'collapsible', 'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(score: 0.89)</br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>', 'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development</strong>: Peach leaf curl first

In [46]:
import random
import re

sender = random.randint(0, 9)

URL     = 'http://localhost:5005/webhooks/rest/webhook'
DATA    = {
    "message"   : "", 
    "sender"    : str(sender)
}
def run_tests(messages):
    results = []

    for m, _ in messages:
        print(m)
        DATA['message'] = m
        r = requests.post(url, json = DATA)
        r = json.loads(r.text)
        
        assert isinstance(r, list)
        assert len(r) >= 3
        r = r[2]

        assert 'custom' in r
        r = r['custom']

        assert 'data' in r
        r = r['data']

        result = []
        for e in r:
            title   = re.findall("<em>(.*?)</em>"           , e['title'])[0]
            link    = re.findall("href=[\"\'](.*?)[\"\']"   , e['title'])[0]
            result.append((title, link))
        
        results.append(result)

        DATA['message'] = '/intent_affirm'
        r = requests.post(url, json = DATA)
        r = json.loads(r.text)

        assert len(r) == 1
        r = r[0]

        assert 'text' in r
        r = r['text']

        assert 'Anything else I can help with?' == r

    return results


In [59]:
import pandas as pd

df = pd.read_csv('./data/UCIPM_Chatbot_Core_Questions 2020_08_19.xlsx - Questions with correct link.csv')

df['request'] = df['Question'   ].apply(lambda x: x.split('\n')[0]              )
df['answer' ] = df['URL'        ].apply(lambda x: x.split('\n')                 )
df['answer' ] = df['answer'     ].apply(lambda x: [x1.split('?')[0] for x1 in x])

df = df[['request', 'answer']]
df.sample(10)

,request,answer
23,How do I kill weeds without killing my lawn?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74113.html]
42,How do you kill head lice?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7446.html]
21,Why is my tree dying?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74109.ht...
29,Should vegetable gardens be planted in a sunny...,[http://ipm.ucanr.edu/PMG/GARDEN/VEGES/CULTURA...
46,An animal is eating all the plants in my garde...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74117.html]
47,I have a garden full of weeds. How do I clear ...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7441.html]
48,What's the best way to get rid of weeds in a l...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7441.html]
38,What do scale insects look like?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7408.html]
7,What attracts roof rats to your house?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74106.html]
3,What is the best way to get rid of peach leaf ...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html]


In [60]:
messages    = df.values.tolist()

results     = run_tests(messages)

How do you treat peach leaf curl?
How can I get rid of peach leaf curl?
If my nectarine tree looks like it has peach leaf curl, how can I cure it?
What is the best way to get rid of peach leaf curl on peach and nectarine trees?
What are the big black bugs on my patio at night?
Are Oriental cockroaches harmful?
Can I get disease from Oriental cockroaches if they get into my home?
What attracts roof rats to your house?
There are roof rats getting into my home. What's the best way to keep them out without using rat poison?
I think I have rats in my home. What kind of rat poison works best? What else can I do to get rid of them?
How do I get rid of ants in my kitchen?
In the summer, I get ant trails all over my kitchen. Why are they coming inside? What will keep them out?
What does rust on roses look like?
My rose leaves are covered with dusty orange spots. Is it rust? What can I do about it?
There are tons of little green bugs on my rose bushes that I think are aphids. Are they bad for my

In [69]:
scores = []
for i, r in enumerate(results):
    print(messages[i][1])
    answers = messages[i][1]
    topn = [False, False, False]
    for i1, r1 in enumerate(r):
        print(f'      {r1[1].split("?")[0]}')
        if r1[1].split('?')[0] in answers:
            if i == 0:
                topn[0] = True
            if i < 3:
                topn[1] = True
            print('score')
            topn[2] = True
    scores.append(topn)

['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html']
      http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html
score
      http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASES/taphlfcrl.html
      http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/wheatcurlmite.html
      http://ipm.ucanr.edu/PMG/GARDEN/FRUIT/PESTS/peachtwigbor.html
      https://www2.ipm.ucanr.edu/Invasive-and-Exotic-Pests/Peach-Root-Knot-Nematode/
['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html']
      http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html
score
      http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASES/taphlfcrl.html
      http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/wheatcurlmite.html
      http://ipm.ucanr.edu/PMG/GARDEN/FRUIT/PESTS/peachtwigbor.html
      https://www2.ipm.ucanr.edu/Invasive-and-Exotic-Pests/Peach-Root-Knot-Nematode/
['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html']
      http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html
score
      https://www2.ipm.ucanr.edu/Invasive-and-Exotic-Pests/Peach-Root-Knot-Nematode/
   

In [73]:
top1 = 0
top3 = 0
top5 = 0
for topn in scores:
    if topn[0]: top1 += 1
    if topn[1]: top3 += 1
    if topn[2]: top5 += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1: {top1:<3d} ({top1/len(results) * 100:<.2f}%)')
print(f'Top 3: {top3:<3d} ({top3/len(results) * 100:<.2f}%)')
print(f'Top 5: {top5:<3d} ({top5/len(results) * 100:<.2f}%)')

Out of 50 results, following correct:
Top 1: 1   (2.00%)
Top 3: 3   (6.00%)
Top 5: 32  (64.00%)


In [67]:
top1

0

In [63]:
top5

32